## 11일 돌린 모델 3개 중 가장 처음에 실행한 걸로 layer 추가 설정해서 돌려보기
* 사유: 손글씨 layer이기 때문에 문제 없을거 같음...부디

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, cv2
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 데이터를 불러오기
# X_loaded shape: (40754, 224, 224, 3)
# y_loaded shape: (40754,)
X = np.load('/content/drive/MyDrive/2.양추정/y값 변경전 X,y np_array/X_data.npy')
y = np.load('/content/drive/MyDrive/2.양추정/y값 변경전 X,y np_array/y_data.npy')

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (40754, 224, 224, 3)
y shape: (40754,)


In [ ]:
X

array([[[[242, 237, 236],
         [242, 237, 236],
         [242, 237, 236],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[242, 237, 236],
         [242, 237, 236],
         [242, 237, 236],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[242, 237, 236],
         [242, 237, 236],
         [242, 237, 236],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [243, 238, 239],
         [247, 241, 242],
         [243, 237, 238]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [241, 236, 237],
         [237, 233, 232],
         [247, 243, 242]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [242, 237, 238],
        

In [ ]:
y

array([1, 1, 1, ..., 5, 5, 5])

In [ ]:
a = np.unique(y)
a

array([1, 2, 3, 4, 5])

In [ ]:
YY = pd.get_dummies(y)
YY

,1,2,3,4,5
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,True,False,False,False,False
4,True,False,False,False,False
...,...,...,...,...,...
40749,False,False,False,False,True
40750,False,False,False,False,True
40751,False,False,False,False,True
40752,False,False,False,False,True


In [ ]:
#split
# train 데이터 크기: (32603, 224, 224, 3) (32603, 5)
# test 데이터 크기: (8151, 224, 224, 3) (8151, 5)
X_train, X_test, y_train, y_test = train_test_split(X, YY, test_size=0.2, shuffle=True, stratify=y, random_state=42)

print("train 데이터 크기:", X_train.shape, y_train.shape)
print("test 데이터 크기:", X_test.shape, y_test.shape)

train 데이터 크기: (32603, 224, 224, 3) (32603, 5)
test 데이터 크기: (8151, 224, 224, 3) (8151, 5)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # 과적합 방지

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # 과적합 방지

#layer Conv2D 추가
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # 과적합 방지

model.add(Dense(5, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#patience=5 검증손실 50번 연속 개선되지 않으면 멈추기
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)

model.fit(X_train, y_train, validation_split=0.25, epochs=100, batch_size=42, verbose=1, callbacks=[early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 54, 54, 64)        0

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
# 모델을 사용하여 테스트 세트의 예측값을 계산
y_pred = model.predict(X_test)

# 예측값 출력
print("처음 30개의 예측값:", y_pred[:30])

255/255 [==============================] - 100s 393ms/step
처음 30개의 예측값: [[8.61756876e-02 2.17077404e-01 2.45622888e-01 1.58934578e-01
  2.92189419e-01]
 [1.18030056e-01 3.87710258e-02 2.39676252e-01 5.67482710e-01
  3.60400118e-02]
 [2.39728093e-01 1.65900573e-01 1.97864786e-01 1.68895006e-01
  2.27611557e-01]
 [1.89929545e-01 2.05264285e-01 2.00022072e-01 2.00138211e-01
  2.04645887e-01]
 [3.75247002e-01 2.17753313e-02 2.49683768e-01 3.39290231e-01
  1.40036475e-02]
 [7.26973712e-02 2.29274064e-01 4.57125455e-01 7.28504434e-02
  1.68052614e-01]
 [1.89929545e-01 2.05264285e-01 2.00022072e-01 2.00138211e-01
  2.04645887e-01]
 [3.86839325e-04 3.61416132e-05 2.09172778e-02 9.76336598e-01
  2.32305657e-03]
 [5.91075003e-01 2.95050681e-01 8.13069269e-02 1.85910277e-02
  1.39762480e-02]
 [3.76652002e-01 9.49822515e-02 2.33146831e-01 1.99778259e-01
  9.54407677e-02]
 [1.06363173e-03 1.47729865e-04 1.39217481e-01 7.64168978e-01
  9.54021811e-02]
 [1.50214946e-02 1.10468343e-02 5.18187648e-03 9



---

## 양분류를 5종 -> 3종만 사용

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, cv2
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# #양분류 카테고리 3개만 가져오기
#사용불가: img resize 한 후 Q1, Q2, Q4 폴더만 사용하려고 했는데 크기가 맞지 않다고 오류 문구발생

# file_path = "/content/drive/MyDrive/YANG/unzip/images/resize"

# data_three = []

# for food_folder in tqdm(os.listdir(file_path)):
#     food_folder_path = os.path.join(file_path, food_folder)

#     for q_folder in os.listdir(food_folder_path):
#         q_folder_path = os.path.join(food_folder_path, q_folder)
#         quantity_n = int(q_folder.split('/')[-1].replace("Q", ""))

#         if q_folder in ["Q1", "Q2", "Q4"]:
#             for img_file in os.listdir(q_folder_path):
#                 if img_file.endswith(".jpg"):
#                     filename = img_file
#                     filepath = os.path.join(q_folder_path, img_file)

#                     data_three.append({"filename": filename, "filepath": filepath, "quantity": quantity_n})

100%|██████████| 83/83 [02:18<00:00,  1.67s/it]


In [ ]:
# #안에 정보 확인하기
# data_three

[{'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00001_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00001_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00002_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00002_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00003_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00003_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00004_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00004_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00005_

In [ ]:
#Q1,2,4 img_resize

file_path = "/content/drive/MyDrive/YANG/unzip/images/resize"
th_resize_path = "/content/drive/MyDrive/YANG/unzip/images/resize_th"

# 각 음식 카테고리의 Q1, Q2, Q4 폴더에 resize된 이미지를 저장
for food_folder in tqdm(os.listdir(file_path)):
    food_folder_path = os.path.join(file_path, food_folder)

    for q_folder in ["Q1", "Q2", "Q4"]:
        q_folder_path = os.path.join(food_folder_path, q_folder)
        save_folder_path = os.path.join(th_resize_path, food_folder, q_folder)

        os.makedirs(save_folder_path, exist_ok=True)  # 저장할 폴더 생성

        # 해당 폴더에 있는 이미지 파일을 resize하여 저장
        for img_file in os.listdir(q_folder_path):
            if img_file.endswith(".jpg"):
                filename = img_file
                filepath = os.path.join(q_folder_path, img_file)

                img = cv2.imread(filepath)
                if img is not None:
                    resized = cv2.resize(img, (224, 224))

                    # 저장할 파일 경로 설정
                    save_file_path = os.path.join(save_folder_path, filename)

                    # 이미지 저장
                    cv2.imwrite(save_file_path, resized)

100%|██████████| 83/83 [1:27:22<00:00, 63.16s/it] 


In [ ]:
#Q1,2,4 img_resize

data_three = []

for food_folder in tqdm(os.listdir(th_resize_path)):
    food_folder_path = os.path.join(th_resize_path, food_folder)

    for q_folder in os.listdir(food_folder_path):
        q_folder_path = os.path.join(food_folder_path, q_folder)
        quantity_n = int(q_folder.split('/')[-1].replace("Q", ""))

        for img_file in os.listdir(q_folder_path):
          if img_file.endswith(".jpg"):
            filename = img_file
            filepath = os.path.join(q_folder_path, img_file)

            data_three.append({"filename": filename, "filepath": filepath, "quantity": quantity_n})

100%|██████████| 83/83 [00:00<00:00, 113.23it/s]


In [ ]:
data_three

[{'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00001_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize_th/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00001_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00002_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize_th/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00002_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00003_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize_th/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00003_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00004_resized.jpg',
  'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize_th/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00004_resized.jpg',
  'quantity': 1},
 {'filename': 'crop_side_건더기국류_뚝배기_갈비ᄐ

In [ ]:
print(data_three[0])

{'filename': 'crop_side_건더기국류_뚝배기_갈비탕_Q1_00001_resized.jpg', 'filepath': '/content/drive/MyDrive/YANG/unzip/images/resize_th/갈비탕/Q1/crop_side_건더기국류_뚝배기_갈비탕_Q1_00001_resized.jpg', 'quantity': 1}


In [ ]:
#data np.array 변환 -> X
X = []

for item in tqdm(data_three):
  img = cv2.imread(item['filepath'])
  X.append(img)

X = np.array(X)
print("X shape:", X.shape)

100%|██████████| 24429/24429 [02:15<00:00, 180.00it/s]


X shape: (24429, 224, 224, 3)


In [ ]:
#data np.array 변환 -> y
y = []

for item in tqdm(data_three):
  y.append(item['quantity'])

y = np.array(y)
print("y shape:", y.shape)

100%|██████████| 24429/24429 [00:00<00:00, 1962247.02it/s]

y shape: (24429,)


In [ ]:
np.save('/content/drive/MyDrive/2.양추정/y값 변경전 X,y np_array/X_data_three.npy', X)
np.save('/content/drive/MyDrive/2.양추정/y값 변경전 X,y np_array/y_data_three.npy', y)

NameError: name 'X' is not defined

In [ ]:
X_three = np.load('/content/drive/MyDrive/2.양추정/y값 변경전 X,y np_array/X_data_three.npy')
y_three = np.load('/content/drive/MyDrive/2.양추정/y값 변경전 X,y np_array/y_data_three.npy')

print("X_loaded shape:", X_three.shape)
print("y_loaded shape:", y_three.shape)

X_loaded shape: (24429, 224, 224, 3)
y_loaded shape: (24429,)


In [ ]:
X_three

array([[[[242, 237, 236],
         [242, 237, 236],
         [242, 237, 236],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[242, 237, 236],
         [242, 237, 236],
         [242, 237, 236],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[242, 237, 236],
         [242, 237, 236],
         [242, 237, 236],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [243, 238, 239],
         [247, 241, 242],
         [243, 237, 238]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [241, 236, 237],
         [237, 233, 232],
         [247, 243, 242]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [242, 237, 238],
        

In [ ]:
y_three

array([1, 1, 1, ..., 4, 4, 4])

In [ ]:
y_uniq = np.unique(y_three)
y_uniq

array([1, 2, 4])

In [ ]:
y_enco = pd.get_dummies(y_three)
y_enco

,1,2,4
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
24424,False,False,True
24425,False,False,True
24426,False,False,True
24427,False,False,True


In [ ]:
#split
X_train, X_test, y_train, y_test = train_test_split(X_three, y_enco, test_size=0.2, shuffle=True, stratify=y_enco, random_state=42)

print("train 데이터 크기:", X_train.shape, y_train.shape)
print("test 데이터 크기:", X_test.shape, y_test.shape)

train 데이터 크기: (19543, 224, 224, 3) (19543, 3)
test 데이터 크기: (4886, 224, 224, 3) (4886, 3)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # 과적합 방지

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # 과적합 방지

#layer Conv2D 추가
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # 과적합 방지

model.add(Dense(3, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint"
checkpoint_path = "model_checkpoint.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#patience=5 검증손실 5번 연속 개선되지 않으면 멈추기
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model.fit(X_train, y_train, validation_split=0.25, epochs=100, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 111, 111, 32)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 54, 54, 64)       

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


349/349 [==============================] - 30s 63ms/step - loss: 7.8335 - accuracy: 0.3301 - val_loss: 1.0987 - val_accuracy: 0.3336
Epoch 2/100
348/349 [============================>.] - ETA: 0s - loss: 1.0996 - accuracy: 0.3358
Epoch 2: val_loss did not improve from 1.09874
349/349 [==============================] - 18s 52ms/step - loss: 1.0996 - accuracy: 0.3362 - val_loss: 1.0993 - val_accuracy: 0.3230
Epoch 3/100
349/349 [==============================] - ETA: 0s - loss: 1.0986 - accuracy: 0.3473
Epoch 3: val_loss did not improve from 1.09874
349/349 [==============================] - 18s 52ms/step - loss: 1.0986 - accuracy: 0.3473 - val_loss: 1.0993 - val_accuracy: 0.3287
Epoch 4/100
349/349 [==============================] - ETA: 0s - loss: 1.0986 - accuracy: 0.3445
Epoch 4: val_loss did not improve from 1.09874
349/349 [==============================] - 18s 52ms/step - loss: 1.0986 - accuracy: 0.3445 - val_loss: 1.0992 - val_accuracy: 0.3330
Epoch 5/100
348/349 [===============

In [ ]:
# 모델 평가하기
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=42)
print('')
print('loss_and_metrics : ' + str(loss_and_metrics))

# 모델 사용하기
xhat_idx = np.random.choice(x_test.shape[0], 5)
xhat = x_test[xhat_idx]
yhat = model.predict_classes(xhat)

for i in range(5):
    print('True : ' + str(argmax(y_test[xhat_idx[i]])) + ', Predict : ' + str(yhat[i]))